In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

def generate_training_data(num_samples=2000):
    """
    Generates a synthetic dataset for training the F1 pit strategy model.
    The logic here helps label the data so the model can learn patterns.
    """
    print("Generating synthetic F1 scenario data...")
    # --- Features ---
    undercut_overcut_opportunity = np.random.choice([0, 1], size=num_samples, p=[0.7, 0.3])
    tire_wear_percentage = np.random.randint(5, 98, size=num_samples)
    performance_drop_seconds = np.random.uniform(0.1, 4.5, size=num_samples) * (tire_wear_percentage / 100)**2
    track_position = np.random.randint(1, 21, size=num_samples)
    race_incident = np.random.choice(
        ['None', 'Yellow Flag', 'Safety Car', 'VSC'],
        size=num_samples,
        p=[0.85, 0.08, 0.04, 0.03]
    )

    data = pd.DataFrame({
        'undercut_overcut_opportunity': undercut_overcut_opportunity,
        'tire_wear_percentage': tire_wear_percentage,
        'performance_drop_seconds': performance_drop_seconds,
        'track_position': track_position,
        'race_incident': race_incident
    })

    # --- Target Variable (The "Correct" Decision Logic) ---
    decisions = []
    for index, row in data.iterrows():
        # High-priority reasons to pit
        if row['race_incident'] in ['Safety Car', 'VSC']:
            decisions.append('PIT NOW')
        elif row['tire_wear_percentage'] > 85 or row['performance_drop_seconds'] > 3.0:
            decisions.append('PIT NOW')
        # Strategic reasons to pit
        elif row['undercut_overcut_opportunity'] == 1 and row['tire_wear_percentage'] > 25 and row['track_position'] < 10:
             decisions.append('PIT NOW')
        elif row['performance_drop_seconds'] > 1.8 and row['tire_wear_percentage'] > 50:
             decisions.append('PIT NOW')
        # Default: Stay out
        else:
            decisions.append('STAY OUT')

    data['decision'] = decisions
    print("Data generation complete.")
    return data

def run_race_simulation(model, preprocessor):
    """
    Runs a mock lap-by-lap F1 race and uses the trained model to make decisions.
    """
    print("\n" + "="*40)
    print("      STARTING RACE SIMULATION")
    print("="*40)

    # Initial Race State
    race_state = {
        'lap': 1,
        'total_laps': 50,
        'undercut_overcut_opportunity': 0,
        'tire_wear_percentage': 5.0,
        'performance_drop_seconds': 0.1,
        'track_position': 3,
        'race_incident': 'None'
    }

    pit_stop_taken = False

    for lap in range(1, race_state['total_laps'] + 1):
        race_state['lap'] = lap

        # --- Update race state dynamically each lap ---
        race_state['tire_wear_percentage'] += np.random.uniform(1.5, 3.5)
        race_state['performance_drop_seconds'] = (race_state['tire_wear_percentage'] / 100)**2 * 4.0 + np.random.uniform(-0.1, 0.1)

        # Randomly introduce incidents or opportunities
        if np.random.rand() < 0.05:
             race_state['race_incident'] = np.random.choice(['Yellow Flag', 'Safety Car', 'VSC'], p=[0.6, 0.2, 0.2])
        else:
             race_state['race_incident'] = 'None'

        if np.random.rand() < 0.15:
             race_state['undercut_overcut_opportunity'] = 1
        else:
             race_state['undercut_overcut_opportunity'] = 0

        # --- Display Current Status ---
        print(f"\n--- Lap {race_state['lap']}/{race_state['total_laps']} ---")
        print(f"Position: P{race_state['track_position']} | Tire Wear: {race_state['tire_wear_percentage']:.1f}% | Perf Drop: {race_state['performance_drop_seconds']:.2f}s")
        if race_state['race_incident'] != 'None':
            print(f"INCIDENT: {race_state['race_incident'].upper()} DEPLOYED!")
        if race_state['undercut_overcut_opportunity'] == 1:
            print("STRATEGY: Undercut/Overcut window is OPEN.")

        # --- Get Model's Decision ---
        # Create a DataFrame for the current state to be processed
        current_state_df = pd.DataFrame([race_state])

        # Ensure correct column order for the model
        current_state_processed = preprocessor.transform(current_state_df)

        decision = model.predict(current_state_processed)[0]

        print("\nMODEL PREDICTION: ")
        print(f"    >> {decision} <<")

        # --- Act on decision ---
        if decision == 'PIT NOW':
            print("\nPIT LANE: Car is coming in! New tires fitted.")
            race_state['tire_wear_percentage'] = 0
            race_state['performance_drop_seconds'] = 0
            positions_lost = np.random.randint(2, 5)
            race_state['track_position'] += positions_lost
            print(f"Lost {positions_lost} positions during the stop.")
            pit_stop_taken = True

        time.sleep(1.5) # Pause to make it readable

    print("\n" + "="*40)
    print("      🏁 CHECKERED FLAG! RACE OVER! 🏁")
    print(f"      Final Position: P{race_state['track_position']}")
    print("="*40)


def main():
    # 1. Generate Data
    dataset = generate_training_data()

    # 2. Define Features (X) and Target (y)
    X = dataset.drop('decision', axis=1)
    y = dataset['decision']

    # 3. Preprocess Data
    # We need to convert the 'race_incident' text field into numbers (One-Hot Encoding)
    categorical_features = ['race_incident']
    one_hot = OneHotEncoder()

    # Create a preprocessor object that applies transformations
    preprocessor = ColumnTransformer(
        [("one_hot", one_hot, categorical_features)],
        remainder="passthrough" # Keep other columns as they are
    )

    # 4. Split Data for Training and Testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 5. Define and Train the Model
    print("Training the Decision Tree model...")
    model = DecisionTreeClassifier(max_depth=7, random_state=42)

    # Fit the preprocessor on training data and transform it
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)

    model.fit(X_train_processed, y_train)
    print("Model training complete.")

    # 6. Evaluate the Model (optional, but good for hackathons)
    accuracy = model.score(X_test_processed, y_test)
    print(f"Model Accuracy on Test Data: {accuracy * 100:.2f}%")

    # 7. Run the live simulation
    run_race_simulation(model, preprocessor)

if __name__ == '__main__':
    main()

Generating synthetic F1 scenario data...
Data generation complete.
Training the Decision Tree model...
Model training complete.
Model Accuracy on Test Data: 100.00%

      STARTING RACE SIMULATION

--- Lap 1/50 ---
Position: P3 | Tire Wear: 7.4% | Perf Drop: -0.02s

MODEL PREDICTION: 
    >> STAY OUT <<

--- Lap 2/50 ---
Position: P3 | Tire Wear: 10.4% | Perf Drop: 0.03s
INCIDENT: YELLOW FLAG DEPLOYED!

MODEL PREDICTION: 
    >> STAY OUT <<

--- Lap 3/50 ---
Position: P3 | Tire Wear: 12.8% | Perf Drop: 0.06s

MODEL PREDICTION: 
    >> STAY OUT <<

--- Lap 4/50 ---
Position: P3 | Tire Wear: 14.6% | Perf Drop: 0.10s

MODEL PREDICTION: 
    >> STAY OUT <<

--- Lap 5/50 ---
Position: P3 | Tire Wear: 17.9% | Perf Drop: 0.07s
STRATEGY: Undercut/Overcut window is OPEN.

MODEL PREDICTION: 
    >> STAY OUT <<

--- Lap 6/50 ---
Position: P3 | Tire Wear: 20.3% | Perf Drop: 0.07s

MODEL PREDICTION: 
    >> STAY OUT <<

--- Lap 7/50 ---
Position: P3 | Tire Wear: 22.5% | Perf Drop: 0.23s
STRATEGY: U

KeyboardInterrupt: 